<a href="https://colab.research.google.com/github/HannaKi/Exploring_StatisticsFinland_zip_code_data/blob/main/zip_code_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
! pip install pyproj

In [2]:
import pandas as pd 
import numpy as np
import json
import pyproj
import plotly.express as px
import pprint

# Data

In [3]:
# https://stat.fi/static/media/uploads/org/avoindata/kartta-aineistojen_lataus_url-osoitteen_kautta_ohje.pdf
!wget "https://geo.stat.fi/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json"

--2022-03-02 20:04:52--  https://geo.stat.fi/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json
Resolving geo.stat.fi (geo.stat.fi)... 193.166.237.56
Connecting to geo.stat.fi (geo.stat.fi)|193.166.237.56|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json’

wfs?service=WFS&ver     [        <=>         ]  23.80M  13.3MB/s    in 1.8s    

2022-03-02 20:04:54 (13.3 MB/s) - ‘wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json’ saved [24957187]



In [4]:
! mv "/content/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json" "paavo2.json"

In [5]:
with open('./paavo2.json', 'r') as json_file:
    p = json.load(json_file)

## Inspect the data

In [6]:
print(type(p))
for k,_ in p.items():
  print(k)

<class 'dict'>
type
features
totalFeatures
numberMatched
numberReturned
timeStamp
crs
bbox


In [7]:
print(p['type'])
print(type(p['features'])) 
print(type(p['features'][0])) # list of dicts --> our features of interest are here
print(p['crs']) # Coordinate reference system --> This is the key for converting the coordinates!
# Nothing interesting here...
#print(p['numberReturned']) 
#print(p['numberMatched']) 
#print(p['timeStamp']) 
#print(p['bbox']) 

FeatureCollection
<class 'list'>
<class 'dict'>
{'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::3067'}}


In [8]:
pp = pprint.PrettyPrinter()
# print the first dictionary in the list of features to see the keys and associated values
pp.pprint(p['features'][0])

{'geometry': {'coordinates': [[[[385653.8928, 6671591.0475],
                                [385573.6195, 6671699.8005],
                                [385612.6593, 6671729.6573],
                                [385575.7769, 6671779.0565],
                                [385548.6877, 6671753.9019],
                                [385510.9652, 6671720.7737],
                                [385461.1218, 6671698.7267],
                                [385390.1366, 6671644.3402],
                                [385345.9799, 6671615.716],
                                [385343.3332, 6671614.8335],
                                [385311.9982, 6671592.797],
                                [385310.7645, 6671592.9281],
                                [385294.653, 6671592.4321],
                                [385222.6021, 6671697.9232],
                                [385291.6376, 6671739.3445],
                                [385184.2491, 6671782.3],
                              

So we have coordinates and demographics. [Data description](https://www.stat.fi/tup/paavo/tietosisalto_ja_esimerkit.html#tietosisalto) (only in Finnish).

In [9]:
# These need to be converted to WGS84 (for all of the items in the feature list)
print(type(p['features'][0]['geometry']['coordinates']))
p['features'][0]['geometry']['coordinates']

<class 'list'>


[[[[385653.8928, 6671591.0475],
   [385573.6195, 6671699.8005],
   [385612.6593, 6671729.6573],
   [385575.7769, 6671779.0565],
   [385548.6877, 6671753.9019],
   [385510.9652, 6671720.7737],
   [385461.1218, 6671698.7267],
   [385390.1366, 6671644.3402],
   [385345.9799, 6671615.716],
   [385343.3332, 6671614.8335],
   [385311.9982, 6671592.797],
   [385310.7645, 6671592.9281],
   [385294.653, 6671592.4321],
   [385222.6021, 6671697.9232],
   [385291.6376, 6671739.3445],
   [385184.2491, 6671782.3],
   [385153.5666, 6671765.4246],
   [385125.016, 6671798.1337],
   [385095.208, 6671799.624],
   [385096.2684, 6671820.8324],
   [385068.1936, 6671830.4498],
   [385055.3828, 6671854.4037],
   [385006.4239, 6671838.7302],
   [384984.6461, 6671864.1748],
   [384776.1724, 6671920.371],
   [384684.0309, 6671919.1363],
   [384507.1821, 6672049.15],
   [384392.6461, 6672053.7862],
   [384294.1154, 6672168.5216],
   [384263.1148, 6672200.5006],
   [384193.8058, 6672207.3336],
   [384170.3668, 667

In [10]:
test=p['features'][0]['geometry']['coordinates'][0][0][0]
print(test)

[385653.8928, 6671591.0475]


For the transformations: 

https://epsg.io/3067: EPSG:3067 equals ETRS89 / TM35FIN(E,N) -- Finland

https://epsg.io/4326: EPSG:4326 equals WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS

In [11]:
# https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
transformer = pyproj.Transformer.from_crs("epsg:3067", "epsg:4326")
transformer.transform(test[0], test[1]) # seems good!

(60.16518005365167, 24.939463163395455)

In [13]:
#paavo=pd.read_csv(r'pno_tilasto.csv', sep=',')
#paavo.head()

In [15]:
for feat in p['features']:
  #print(feat['id'])
  #print(feat['geometry']['coordinates'])
  
  for c in feat['geometry']['coordinates']:
    wgs84_list=[]
    for i in c:
      #print(i)
      for j in i:
        
        wgs84=transformer.transform(j[0], j[1])
        wgs84=list(wgs84)
        print(j,"-->",wgs84)
        wgs84_list.append(wgs84)
    print(wgs84_list)
  break
    #print(c[0][0])
 
#hhhh
#for coordinate in 







[385653.8928, 6671591.0475] --> [60.16518005365167, 24.939463163395455]
[385573.6195, 6671699.8005] --> [60.166133429256554, 24.937956555856044]
[385612.6593, 6671729.6573] --> [60.1664122805203, 24.938642761757187]
[385575.7769, 6671779.0565] --> [60.16684521525895, 24.93795082881654]
[385548.6877, 6671753.9019] --> [60.16661190454415, 24.937477172597546]
[385510.9652, 6671720.7737] --> [60.166304061714655, 24.936816534335954]
[385461.1218, 6671698.7267] --> [60.16609224935875, 24.93593140943601]
[385390.1366, 6671644.3402] --> [60.16558430816215, 24.934683815033484]
[385345.9799, 6671615.716] --> [60.16531506013388, 24.933904827862914]
[385343.3332, 6671614.8335] --> [60.16530639805535, 24.933857667182096]
[385311.9982, 6671592.797] --> [60.16509985964611, 24.933305852951094]
[385310.7645, 6671592.9281] --> [60.16510068949293, 24.93328356450279]
[385294.653, 6671592.4321] --> [60.16509171282555, 24.93299373332569]
[385222.6021, 6671697.9232] --> [60.166018055890696, 24.9316368444193]

In [ ]:
with open('./paavo_polygons.json', 'r') as json_file:
    p = json.load(json_file)

In [ ]:
def plot_map(df): 
    fig = px.scatter_mapbox(
        df, 
        lat='lat', 
        lon='lon', 
        # size='size',
        hover_name='postinumeroalue', 
        hover_data=['postinumeroalue'],
        color='postinumeroalue', 
        zoom=11, 
        height=1000)
    fig.update_layout(mapbox_style='open-street-map')
    fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
    fig.show(renderer='notebook') # Needed for animations in VS Code # https://community.plotly.com/t/animation-not-working-in-jupyter-notebook-in-vscode/38021
    # fig.show()

plot_map(paavo)